In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import matplotlib
import matplotlib.pyplot as plt

import torch
from facenet_pytorch import MTCNN, InceptionResnetV1
from PIL import Image

In [2]:
mtcnn =  MTCNN(image_size=240, keep_all=True, min_face_size=40, )
resnet = InceptionResnetV1(pretrained="vggface2")

resnet = resnet.eval()

print(f"MTCNN image size: {mtcnn.image_size}")
print(f"MTCNN keeping all faces: {mtcnn.keep_all}")
print(f"InceptionResnet weight set: {resnet.pretrained}")

MTCNN image size: 240
MTCNN keeping all faces: True
InceptionResnet weight set: vggface2


In [3]:
# We'll also need the library of known faces
# Load the `embeddings.pt` file with `torch`
embedding_data = torch.load("embeddings.pt")

print(f"Known names: {[data[1] for data in embedding_data]}")

Known names: ['mary_kom', 'ranveer']


In [4]:
# We'll be getting images uploaded to our app. 
# But we'll need to test things as we go, so let's get a few sample images.
project_dir = Path("project4")
images_dir = project_dir / "data" / "extracted_frames"

print(images_dir)

project4\data\extracted_frames


In [10]:
from pathlib import Path
images_dir = Path("project4") / "data" / "extracted_frames"

In [16]:
# Here are two sample images to test as we go.
sample_single = Image.open(images_dir / "frame_10.jpg")
sample_multiple = Image.open(images_dir / "frame_1280.jpg")

FileNotFoundError: [Errno 2] No such file or directory: 'project4\\data\\extracted_frames\\frame_10.jpg'

In [7]:
sample_single


NameError: name 'sample_single' is not defined

In [13]:
sample_multiple

NameError: name 'sample_multiple' is not defined

In [14]:
#  get the cropped images and probabilities, and the bounding boxes
def locate_faces(image):
    cropped_images, probs = mtcnn(image, return_prob=True)
    boxes, _ = mtcnn.detect(image)

    if boxes is None or cropped_images is None:
        return []
    else:
        return list(zip(boxes, probs, cropped_images))

In [15]:
# do a test run on the `sample_multiple`
multiple_faces = locate_faces(sample_multiple)
print(f"How many faces in the sample with 5 faces: {len(multiple_faces)}")

NameError: name 'sample_multiple' is not defined

In [11]:
face = multiple_faces[0]
print(f"First bounding box: {face[0]}")
print(f"First probability: {face[1]}")
print(f"Shape of first cropped image: {face[2].shape}")

NameError: name 'multiple_faces' is not defined

In [12]:
def determine_name_dist(cropped_image, threshold=0.9):
    # Use `resnet` on `cropped_image` to get the embedding.
    # Don't forget to unsqueeze!
    emb = resnet(cropped_image.unsqueeze(0))

    # We'll compute the distance to each known embedding
    distances = []
    for known_emb, name in embedding_data:
        # Use torch.dist to compute the distance between
        # `emb` and the known embedding `known_emb`
        dist = torch.dist(emb, known_emb).item()
        distances.append((dist, name))

    # Find the name corresponding to the smallest distance
    dist, closest = min(distances)

    # If the distance is less than the threshold, set name to closest
    # otherwise set name to "Undetected"
    if dist < threshold:
        name = closest
    else:
        name = "Undetected"

    return name, dist

In [13]:
# And to see if it worked, let's run it on our faces.
print("Who's in the picture with 5 faces, with distances?")
for index, face in enumerate(multiple_faces):
    print(f"{index}: {determine_name_dist(face[2])}")

Who's in the picture with 5 faces, with distances?


NameError: name 'multiple_faces' is not defined

In [14]:
def label_face(name, dist, box, axis):
    """Adds a box and a label to the axis from matplotlib
    - name and dist are combined to make a label
    - box is the four corners of the bounding box for the face
    - axis is the return from fig.subplots()
    Call this in the same cell as the figure is created"""

    # Add the code to generate a Rectangle for the bounding box
    # set the color to "blue" and fill to False
    rect = plt.Rectangle(
            (box[0], box[1]),
            box[2] - box[0],
            box[3] - box[1],
            fill=False,
            color="blue",
            )

    axis.add_patch(rect)

    # Set color to be red if the name is "Undetected"
    # otherwise set it to be blue
    if name == "Undetected":
        color = "red"
    else:
        color= "blue"
    label = f"{name} {dist:.2f}"
    axis.text(box[0], box[1], label, fontsize="large", color=color)

In [15]:
# To demonstrate how it works, we'll plot the first face found in the multiple faces
# This sets the image size
# and draws the original image
width, height = sample_multiple.size
dpi = 96
fig = plt.figure(figsize=(width / dpi, height / dpi), dpi=dpi)
axis = fig.subplots()
axis.imshow(sample_multiple)
plt.axis("off")

face = multiple_faces[0]
cropped_image = face[2]
box = face[0]

name, dist = determine_name_dist(cropped_image)

label_face(name, dist, box, axis)

NameError: name 'sample_multiple' is not defined

In [16]:
width, height = sample_multiple.size
dpi = 96
fig = plt.figure(figsize=(width / dpi, height / dpi), dpi=dpi)
axis = fig.subplots()
axis.imshow(sample_multiple)
plt.axis("off")

NameError: name 'sample_multiple' is not defined

In [17]:
len(multiple_faces[0])

NameError: name 'multiple_faces' is not defined

In [18]:
# Fill in the needed loop to go over the faces in `multiple_faces`.
# This sets the image size
# and draws the original image
width, height = sample_multiple.size
dpi = 96
fig = plt.figure(figsize=(width / dpi, height / dpi), dpi=dpi)
axis = fig.subplots()
axis.imshow(sample_multiple)
plt.axis("off")

for face in multiple_faces:
    box, prob, cropped_image = face

    name, dist = determine_name_dist(cropped_image)

    label_face(name, dist, box, axis)

NameError: name 'sample_multiple' is not defined

In [19]:
def add_labels_to_image(image):
    # This sets the image size
    # and draws the original image
    width, height = image.width, image.height
    dpi = 96
    fig = plt.figure(figsize=(width / dpi, height / dpi), dpi=dpi)
    axis = fig.subplots()
    axis.imshow(image)
    plt.axis("off")

    # Use the function locate_faces to get the individual face info
    faces = locate_faces(image)

    for box, prob, cropped in faces:
        # If the probability is less than 0.90,
        # It's not a face, skip this run of the loop with continue
        if prob < 0.9:
            continue
        
        # Call determine_name_dist to get the name and distance
        name, dist = determine_name_dist(cropped)

        # Use label_face to draw the box and label on this face
        label_face(name, dist, box, axis)

    return fig

In [20]:
# We can test this by running it on our `sample_multiple` (the original image). 
labeled_multiple = add_labels_to_image(sample_multiple)

NameError: name 'sample_multiple' is not defined

In [21]:
labeled_multiple

NameError: name 'labeled_multiple' is not defined

In [22]:
#Call our `add_labels_to_image` function on `sample_single` 
# and save the result to `labeled_single`.
labeled_single = add_labels_to_image(sample_single)

NameError: name 'sample_single' is not defined

In [23]:
labeled_single

NameError: name 'labeled_single' is not defined

In [24]:
import face_recognition

ModuleNotFoundError: No module named 'face_recognition'

In [25]:
face_recognition

NameError: name 'face_recognition' is not defined

In [26]:
test_multiple = face_recognition.add_labels_to_image(sample_multiple)

NameError: name 'face_recognition' is not defined

In [27]:
test_multiple

NameError: name 'test_multiple' is not defined

In [28]:
import app

ModuleNotFoundError: No module named 'app'

In [29]:
assert hasattr(app, "add_labels_to_image"), "import not successful"

NameError: name 'app' is not defined

In [30]:
# Tests if the image processing worked
# you'll get an error if it didn't
f = open("project4/data/images/mary_kom/frame_115.jpg", "rb")
res = app.run_face_recognition(f)
f.close()

assert isinstance(res, matplotlib.figure.Figure), "Image did not process"

FileNotFoundError: [Errno 2] No such file or directory: 'project4/data/images/mary_kom/frame_115.jpg'